In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [3]:
from sklearn.metrics import get_scorer_names
get_scorer_names()

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_negative_likelihood_ratio',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'positive_likelihood_ratio',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',

In [2]:
from sklearn.datasets import make_moons

dados = make_moons(n_samples=10000,noise=0.4)
dados

(array([[ 0.84762449, -0.80761547],
        [ 0.02524715,  1.39014759],
        [ 2.61720047,  0.78302577],
        ...,
        [-0.58209353,  0.78709277],
        [ 1.10714289, -0.57279666],
        [-1.10085615,  1.33044874]]),
 array([1, 0, 1, ..., 0, 1, 0], dtype=int64))

In [5]:
from sklearn.tree import DecisionTreeClassifier
dados[0]

array([[ 0.84762449, -0.80761547],
       [ 0.02524715,  1.39014759],
       [ 2.61720047,  0.78302577],
       ...,
       [-0.58209353,  0.78709277],
       [ 1.10714289, -0.57279666],
       [-1.10085615,  1.33044874]])

In [6]:
dados[1]

array([1, 0, 1, ..., 0, 1, 0], dtype=int64)

In [7]:
X = dados[0]
y = dados[1]

In [8]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [14]:
from sklearn.model_selection import GridSearchCV
tree = DecisionTreeClassifier()
param_grid = {
    "max_leaf_nodes":[2,4,6,8,10,11,12,13,14,15,16,17,18,19,20,21,22],
    "max_depth":[1,2,3,4,5,6,7,8,9,10],
    "min_samples_leaf":[1,2,3,4,5,6,7,8,9,10,11,12,13]
}

model_final = GridSearchCV(tree,param_grid,cv = 3,scoring="accuracy")
model_final.fit(x_train,y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'max_leaf_nodes': [2, 4, 6, 8, 10, 11, 12, 13, 14, 15,
                                            16, 17, 18, 19, 20, 21, 22],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                              12, 13]},
             scoring='accuracy')

In [16]:
final_model = model_final.best_estimator_
final_model

DecisionTreeClassifier(max_depth=7, max_leaf_nodes=22, min_samples_leaf=9)

In [18]:
from sklearn.model_selection import cross_val_score

cross_val_score(final_model,x_test,y_test,cv=3,scoring="accuracy")

array([0.85307346, 0.84707646, 0.85135135])

In [36]:
from sklearn.ensemble import VotingClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression

from sklearn.svm import SVC

svc = SVC(probability=True)
random = RandomForestClassifier()
logistic = LogisticRegression()
votting = VotingClassifier(estimators=[("svc",svc),("random",random),("logistic",logistic),("tree",final_model)],voting = "soft")

In [37]:
votting.fit(x_train,y_train)

VotingClassifier(estimators=[('svc', SVC(probability=True)),
                             ('random', RandomForestClassifier()),
                             ('logistic', LogisticRegression()),
                             ('tree',
                              DecisionTreeClassifier(max_depth=7,
                                                     max_leaf_nodes=22,
                                                     min_samples_leaf=9))],
                 voting='soft')

In [38]:
y_predict = votting.predict(x_test)

from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_predict)

0.8675